In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Thu Jun 11 11:29:16 PDT 2020


# Check model 6 NTD BP

In [2]:
output_dirs = ['/ihme/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/v6.0_prelim_maternal_iron_lri_bp/india_5_20/count_data/']

locations = ['India']

In [3]:
# available data frames

# births
# births_with_ntd
# birth_weight
# deaths
# gestational_age
# person_time
# population
# state_person_time
# transition_count
# ylds
# ylls

In [4]:
births = pd.read_hdf(output_dirs[0] + 'births.hdf')
births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births,29,baseline,327.0
1,2020,female,covered,live_births,29,baseline,5.0
2,2020,female,covered,live_births,29,baseline,80.0
3,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,233.0
4,2020,female,covered,live_births,29,folic_acid_fortification_scale_up,4.0


In [21]:
births_by_scenario = births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex').mean()
births_by_scenario.head()

value
scenario                          sex            
baseline                          female  42780.0
                                  male    47090.6
folic_acid_fortification_scale_up female  38396.0
                                  male    42065.0
vitamin_a_fortification_scale_up  female  48467.4

In [8]:
ntd_births = pd.read_hdf(output_dirs[0] + 'births_with_ntd.hdf')
ntd_births.head()

,year,sex,folic_acid_fortification_group,measure,input_draw,scenario,value
0,2020,female,covered,live_births_with_ntds,29,baseline,0.0
1,2020,female,covered,live_births_with_ntds,29,baseline,0.0
2,2020,female,covered,live_births_with_ntds,29,baseline,0.0
3,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0
4,2020,female,covered,live_births_with_ntds,29,folic_acid_fortification_scale_up,0.0


In [22]:
ntd_births_by_scenario = ntd_births.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
ntd_births_by_scenario

value
scenario                          sex          
baseline                          female   21.7
                                  male     18.7
folic_acid_fortification_scale_up female   19.7
                                  male     16.7
vitamin_a_fortification_scale_up  female   22.0
                                  male     20.7

In [23]:
ntd_bp = ntd_births_by_scenario / births_by_scenario
ntd_bp

value
scenario                          sex             
baseline                          female  0.000507
                                  male    0.000397
folic_acid_fortification_scale_up female  0.000513
                                  male    0.000397
vitamin_a_fortification_scale_up  female  0.000454
                                  male    0.000388

In [24]:
c = get_ids('cause')
c[c['cause_name'].str.contains('eural tube')]

,cause_id,cause_name,acause,cause_description
348,642,Neural tube defects,cong_neural,None


# Check model 4 NTD BP using same methodology

In [29]:
output_dir4 = '/share/costeffectiveness/results/vivarium_conic_lsff/v4.0_folic_acid_fortification/india/2020_03_18_23_04_36/count_data/'

In [35]:
births4 = pd.read_hdf(output_dir4 + 'births.hdf')
births4 = births4.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
births4.head()

value
scenario                          sex              
baseline                          female  162779.48
                                  male    179124.68
folic_acid_fortification_scale_up female  162779.48
                                  male    179124.68

In [36]:
ntd_births4 = pd.read_hdf(output_dir4 + 'births_with_ntd.hdf')
ntd_births4 = ntd_births4.groupby(['input_draw','scenario','sex']).sum().groupby(['scenario','sex']).mean()
ntd_births4.head()

value
scenario                          sex           
baseline                          female  122.76
                                  male    123.80
folic_acid_fortification_scale_up female  119.80
                                  male    121.24

In [37]:
bp4 = ntd_births4 / births4
bp4

value
scenario                          sex             
baseline                          female  0.000754
                                  male    0.000691
folic_acid_fortification_scale_up female  0.000736
                                  male    0.000677

# Check artifact NTD BP

In [39]:
art = Artifact('/share/costeffectiveness/artifacts/vivarium_conic_lsff/india.hdf', 
                   filter_terms=['year_start == 2017', 'age_start < 5'])
data = art.load('cause.neural_tube_defects.birth_prevalence')
data_mean = pd.DataFrame(data.mean(axis=1)).rename(columns={0:'value'})
data_limits = pd.DataFrame(data.quantile([0.025,0.975], axis=1)).transpose().rename(columns={0.025:'lower',
                                                                                             0.975:'upper'})
data = data_mean.merge(data_limits, right_index=True, left_index=True).reset_index()
data.head()

,location,sex,year_start,year_end,value,lower,upper
0,India,Female,2017,2018,0.000762,0.000678,0.000852
1,India,Male,2017,2018,0.000714,0.000636,0.000801


# Conclusion:

- Model 4 NTD BP was within GBD range, Model 6 NTD BP was below GBD range
- Model 4 had same number of live births for each scenario, model 6 did not